In [ ]:
path="/content/drive/MyDrive/Dataset/Clinical_Test"

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
train_clinical=pd.read_csv("/content/drive/MyDrive/Dataset/Clinical_Test/train_clinical_data.csv")

In [ ]:
train_proteins=pd.read_csv("/content/drive/MyDrive/Dataset/Clinical_Test/train_proteins.csv")

In [ ]:
train_peptides=pd.read_csv("/content/drive/MyDrive/Dataset/Clinical_Test/train_peptides.csv")

In [ ]:
train_clinical.head()

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.0,6.0,15.0,NaN,NaN
1,55_3,55,3,10.0,7.0,25.0,NaN,NaN
2,55_6,55,6,8.0,10.0,34.0,NaN,NaN
3,55_9,55,9,8.0,9.0,30.0,0.0,On
4,55_12,55,12,10.0,10.0,41.0,0.0,On


In [ ]:
train_proteins.head()

,visit_id,visit_month,patient_id,UniProt,NPX
0,55_0,0,55,O00391,11254.3
1,55_0,0,55,O00533,732430.0
2,55_0,0,55,O00584,39585.8
3,55_0,0,55,O14498,41526.9
4,55_0,0,55,O14773,31238.0


In [ ]:
train_peptides.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [ ]:
def prepare_dataset(train_proteins, train_peptides):
    # Group and calculate mean for proteins
    protein_grouped = train_proteins.groupby(['visit_id', 'UniProt'])['NPX'].mean().reset_index()

    # Group and calculate mean for peptides
    peptide_grouped = train_peptides.groupby(['visit_id', 'Peptide'])['PeptideAbundance'].mean().reset_index()

    # Pivot the grouped dataframes
    protein_pivoted = protein_grouped.pivot(index='visit_id', columns='UniProt', values='NPX').rename_axis(columns=None).reset_index()
    peptide_pivoted = peptide_grouped.pivot(index='visit_id', columns='Peptide', values='PeptideAbundance').rename_axis(columns=None).reset_index()

    # Merge the pivoted dataframes on 'visit_id'
    pro_pep_df = protein_pivoted.merge(peptide_pivoted, on='visit_id', how='left')

    # Return the final merged dataframe
    return pro_pep_df


In [ ]:
pro_pep_df=prepare_dataset(train_proteins, train_peptides)

In [ ]:
pro_pep_df.head()

,visit_id,O00391,O00533,O00584,O14498,O14773,O14791,O15240,O15394,O43505,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,10053_0,9104.27,402321.0,NaN,NaN,7150.57,2497.84,83002.9,15113.6,167327.0,...,202274.0,NaN,4401830.0,77482.6,583075.0,76705.7,104260.0,530223.0,NaN,7207.30
1,10053_12,10464.20,435586.0,NaN,NaN,NaN,NaN,197117.0,15099.1,164268.0,...,201009.0,NaN,5001750.0,36745.3,355643.0,92078.1,123254.0,453883.0,49281.9,25332.80
2,10053_18,13235.70,507386.0,7126.96,24525.7,NaN,2372.71,126506.0,16289.6,168107.0,...,220728.0,NaN,5424380.0,39016.0,496021.0,63203.6,128336.0,447505.0,52389.1,21235.70
3,10138_12,12600.20,494581.0,9165.06,27193.5,22506.10,6015.90,156313.0,54546.4,204013.0,...,188362.0,9433.71,3900280.0,48210.3,328482.0,89822.1,129964.0,552232.0,65657.8,9876.98
4,10138_24,12003.20,522138.0,4498.51,17189.8,29112.40,2665.15,151169.0,52338.1,240892.0,...,206187.0,6365.15,3521800.0,69984.6,496737.0,80919.3,111799.0,NaN,56977.6,4903.09


In [ ]:
def split_dataset(dataset,test_ratio=0.20):
  test_indices=np.random.rand(len(dataset))<test_ratio
  return dataset[~test_indices],dataset[test_indices]

In [ ]:
!pip install tensorflow_decision_forests

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 MB 920.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 29.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.0
    Uninstalling ml-dtypes-0.4.0:
      Successfully uninstalled ml-dtypes-0.4.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: tensorflow
    Found ex

In [ ]:
import tensorflow_decision_forests as tfdf

In [ ]:
tfdf.keras.get_all_models()

[tensorflow_decision_forests.keras.RandomForestModel,
 tensorflow_decision_forests.keras.GradientBoostedTreesModel,
 tensorflow_decision_forests.keras.CartModel,
 tensorflow_decision_forests.keras.DistributedGradientBoostedTreesModel]

In [ ]:
def smape(A, F):
  return 100/len(A)* np.sum(2*np.abs(F-A)/(np.abs(A)+np.abs(F)))

In [ ]:
model_dict={}

mse_dict={}
smape_dict={}

In [ ]:
FEATURES=[I for I in pro_pep_df.columns if I not in ["visit_id"]]
FEATURES.append("visit_month")

In [ ]:
target=["updrs_1","updrs_2","updrs_3","updrs_4"]

In [ ]:
for label in target:
    dataset_df = pro_pep_df.merge(train_clinical[['visit_id', 'patient_id', 'visit_month', label]], on=['visit_id'], how='left')
    dataset_df = dataset_df.dropna(subset=[label])

    feature_list = FEATURES.copy()
    feature_list.append(label)

    train_df, valid_df = split_dataset(dataset_df[feature_list])
    train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label=label, task=tfdf.keras.Task.REGRESSION)
    valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_df, label=label, task=tfdf.keras.Task.REGRESSION)

    rf = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION, verbose=0)
    rf.compile(metrics=["mse"])
    rf.fit(x=train_ds)

    model_dict[label] = rf

    inspector = rf.make_inspector()
    inspector.evaluation()
    evaluation = rf.evaluate(x=valid_ds, return_dict=True)
    mse_dict[label] = evaluation["mse"]

    preds = rf.predict(valid_ds)
    smape_dict[label] = smape(valid_df[label].values.tolist(), preds.flatten())


1/1 [==============================] - 3s 3s/step


1/1 [==============================] - 4s 4s/step - loss: 0.0000e+00 - mse: 205.7738


1/1 [==============================] - 2s 2s/step


In [ ]:
for name, value in mse_dict.items():
    print(f"{name}: {value}:.4f")

print("\n")